In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import csv
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import metrics
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.decomposition import LatentDirichletAllocation

df = pd.read_csv('../dados_categorizados_completo_novo.csv')
df.head()

,Id,Enunciado,Tópico,Contexto
0,994,Meu primeiro programa\nEscreva um programa que...,Ambientação,Outros
1,996,Impressão de caracteres na tela\nEscreva um pr...,Ambientação,Outros
2,999,Impressão de caracteres na tela (Bart Simpson)...,Ambientação,Outros
3,1000,Operadores aritméticos\nQual o valor de X para...,Ambientação,Matemático
4,578,Operadores aritméticos \nQual o valor de Y par...,Ambientação,Matemático


In [2]:
df.Contexto.value_counts()

Matemático         261
Comercial          120
Jogos               96
Escolar             79
Outros              59
Trânsito            43
Esporte             42
Física              36
Bancário            35
RH                  35
Filmes e séries     30
População           25
Química             23
Pessoa              22
Data e hora         21
Segurança           20
Pesquisa            18
Meio ambiente       18
Saúde               17
Consumo             16
Geografia           11
Produção             7
Computacional        6
Imposto              5
Name: Contexto, dtype: int64

In [3]:
import spacy
nlp = spacy.load('pt')

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
X = df['Enunciado']
y = df['Contexto']

In [5]:
class LemmaTokenizer(object):
    def __call__(self, text):
        doc=nlp(text)
        tokens=[]
        for token in doc:
            if(token.text.isalpha() and len(token.text)>=2):
                tokens+=[token]
        return [t.lemma_ for t in tokens if t not in nlp.Defaults.stop_words]
pipeline= LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    dual=True,
    tol=0.001,
    C=10.0,
    multi_class='ovr',
    fit_intercept=True,
    intercept_scaling=10,
    class_weight=None,
    verbose=0,
    random_state=None,
    max_iter=10000,
)

In [6]:
# from sklearn.model_selection import StratifiedKFold
# vetor_X_train=[]
# vetor_y_train=[]
# vetor_X_test=[]
# vetor_y_test=[]
# split = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
# for train_index, val_index in split.split(X, y):
#     vetor_X_train+=[X.iloc[train_index]]
#     vetor_X_test+=[X.iloc[val_index]]
#     vetor_y_train+=[y[train_index]]
#     vetor_y_test+=[y[val_index]]

In [7]:
# contador=0
# while(contador<len(vetor_X_train)):
#     X_train=vetor_X_train[contador]
#     y_train=vetor_y_train[contador]
#     X_test=vetor_X_test[contador]
    
#     text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=nlp.Defaults.stop_words,
#                             tokenizer=LemmaTokenizer(),sublinear_tf=True)),
#                             ('svd', TruncatedSVD(algorithm='randomized', n_components=300)),
#                             ('clf',pipeline),
#     ])
#     text_clf_lsvc2.fit(X_train, y_train)


#     contador+=1

In [8]:
text_clf_lsvc2 = Pipeline([('tfidf', TfidfVectorizer(stop_words=nlp.Defaults.stop_words,
                            tokenizer=LemmaTokenizer(),sublinear_tf=True)),
                            ('svd', TruncatedSVD(algorithm='randomized', n_components=300)),
                            ('clf',pipeline),
    ])
text_clf_lsvc2.fit(X, y)

/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['acercar', 'algum', 'antar', 'apoiar', 'baixar', 'caminhar', 'ceder', 'cincar', 'comer', 'conhecer', 'custar', 'dever', 'devir', 'direito', 'entrar', 'estevar', 'estivar', 'falto', 'formar', 'grupar', 'irar', 'ligar', 'mear', 'nado', 'nenhum', 'nó', 'o', 'obrigar', 'oitavar', 'orar', 'outro', 'parecer', 'parir', 'pelar', 'pontar', 'pôr', 'quantum', 'quartar', 'querer', 'quietar', 'quintar', 'segundar', 'seriar', 'sobrar', 'suar', 'tardar', 'terceirar', 'umar', 'vezar', 'vir', 'vário'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/samuel/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words={'acerca', 'ademais', 'adeus',
                                             'agora', 'ainda', 'algo',
                                             'alguma...
                                 tokenizer=<__main__.LemmaTokenizer object at 0x7f92d1c4d150>,
                                 use_idf=True, vocabulary=None)),
                ('svd',
                 TruncatedSVD(algorithm='randomized', n_components=300,
                  

In [9]:
cv=CountVectorizer(max_df=0.5,min_df=5,stop_words=nlp.Defaults.stop_words)
dtm=cv.fit_transform(df['Enunciado'])
LDA=LatentDirichletAllocation(n_components=1100,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=1100, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [10]:
teste=[]
for i,topic in enumerate(LDA.components_):
    teste+=[' '.join([cv.get_feature_names()[index] for index in topic.argsort()[-25:]])]
predictions=text_clf_lsvc2.predict(teste)

In [11]:
predictions

array(['Comercial', 'Comercial', 'Matemático', ..., 'Comercial', 'Outros',
       'Comercial'], dtype=object)

In [12]:
topic_results=LDA.transform(dtm)

In [13]:
y_pred=[]
for elemento in topic_results.argmax(axis=1):
    y_pred+=[predictions[elemento]]


In [14]:
df['LDA']=y_pred

In [15]:
df.LDA.value_counts()

Matemático         272
Comercial          123
Jogos               95
Escolar             71
Outros              46
Esporte             44
Trânsito            38
Filmes e séries     37
Física              35
Bancário            34
RH                  34
População           25
Pessoa              24
Química             22
Segurança           21
Meio ambiente       20
Data e hora         20
Consumo             18
Saúde               18
Pesquisa            16
Produção            15
Geografia            7
Imposto              6
Computacional        4
Name: LDA, dtype: int64

In [16]:
df.Contexto.value_counts()

Matemático         261
Comercial          120
Jogos               96
Escolar             79
Outros              59
Trânsito            43
Esporte             42
Física              36
Bancário            35
RH                  35
Filmes e séries     30
População           25
Química             23
Pessoa              22
Data e hora         21
Segurança           20
Pesquisa            18
Meio ambiente       18
Saúde               17
Consumo             16
Geografia           11
Produção             7
Computacional        6
Imposto              5
Name: Contexto, dtype: int64

In [17]:
y_test=df['Contexto']
y_pred=df['LDA']

In [18]:
print(metrics.confusion_matrix(y_test,y_pred))

[[ 31   3   0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  1  98   0   2   0   0   2   0   0   0   1   1   6   0   0   0   1   0
    6   0   1   1   0   0]
 [  0   0   4   0   0   0   0   0   2   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0  16   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0  19   0   0   0   0   0   0   0   1   0   0   0   0   0
    1   0   0   0   0   0]
 [  0   4   0   0   0  68   2   2   0   0   0   0   3   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0  40   2   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0  28   0   0   0   0   0   2   0   0   0   0
    0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30   0   0   1   4   0   0   0   0   0
    0   0   0   0   0   1]
 [  0   0   0   0   0   0   0   3   0   7   0   0   1   0   0   0   0   0
    0   0   0   0

In [19]:
print(metrics.classification_report(y_test,y_pred))

                 precision    recall  f1-score   support

       Bancário       0.91      0.89      0.90        35
      Comercial       0.80      0.82      0.81       120
  Computacional       1.00      0.67      0.80         6
        Consumo       0.89      1.00      0.94        16
    Data e hora       0.95      0.90      0.93        21
        Escolar       0.96      0.86      0.91        79
        Esporte       0.91      0.95      0.93        42
Filmes e séries       0.76      0.93      0.84        30
         Física       0.86      0.83      0.85        36
      Geografia       1.00      0.64      0.78        11
        Imposto       0.83      1.00      0.91         5
          Jogos       0.93      0.92      0.92        96
     Matemático       0.90      0.94      0.92       261
  Meio ambiente       0.90      1.00      0.95        18
         Outros       0.89      0.69      0.78        59
       Pesquisa       1.00      0.89      0.94        18
         Pessoa       0.83    

In [20]:
print(metrics.accuracy_score(y_test,y_pred))

0.8928229665071771
